In [1]:
import sqlalchemy
import pandas as pd
import torch as T
from torch.utils.data import Dataset


class SqliteDataSet(Dataset):
    def __init__(self,
                 sqlite_rul='sqlite:///D:\\sqlite\\data\\finance.db',
                 query='select * from STOCK_DAILY',
                 transform=None):

        self.engine = sqlalchemy.create_engine('sqlite:///D:\\sqlite\\data\\finance.db')
        self.transform = transform
        self.result_set = pd.read_sql_query(query,self.engine.connect())
        self.length = self.result_set.shape[0]

    def __getitem__(self, index) -> T.Tuple[T.Tensor,int]:
        data = self.result_set.loc[index]
        rate = ( data['Y_CLOSE5_MAX'] - data['Close'] ) / data['Close'] * 100
        x = T.tensor(data.drop('Y_CLOSE5_MAX')).to(T.float32)        
        if rate > 5:
            y = 5
        elif 2 < rate < 5:
            y = 4
        elif rate < -5:
            y = 1
        elif rate < -2:
            y = 2
        else:
            y = 3          
       
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return self.length

In [2]:
import sqlalchemy
import pandas as pd
import torch as T
query =r"""select Open ,High ,Low ,Close ,"Adj Close" ,Volume ,MACD_12_26_9 ,MACDh_12_26_9 ,MACDs_12_26_9 ,RSI_14 ,"BBL_5_2.0" ,"BBM_5_2.0" ,"BBU_5_2.0" ,"BBB_5_2.0" ,"BBP_5_2.0" ,OBV ,SMA_20 ,EMA_50 ,STOCHk_14_3_3 ,STOCHd_14_3_3 ,ADX_14 ,DMP_14 ,DMN_14 ,AD ,STDEV_30 ,VWAP_D ,PV_DERIVATIVE ,Y_CLOSE5_MAX from STOCK_DAILY WHERE ticker='TSLA'"""
dataset = SqliteDataSet(query=query)
data = dataset.__getitem__(0)
print(data)

# the_engine = sqlalchemy.create_engine('sqlite:///D:\\sqlite\\data\\finance.db')
# query = 'select * from STOCK_DAILY where ticker=\'MARA\''
# with the_engine.connect() as con:
#     result = pd.read_sql_query(query,con)
#     print(type(result))
#     result = result.drop('Date',axis=1)
#     result = result.drop('ticker',axis=1)
# 
#     data = result.loc[100]
#     print(type(data))
#     x = T.tensor(data.drop('Y_CLOSE5_MAX'))
#     rate = ( data['Y_CLOSE5_MAX'] - data['Close'] ) / data['Close'] * 100
#     if rate > 5:
#         y = 5
#     elif 2 < rate < 5:
#         y = 4
#     elif rate < -5:
#         y = 1
#     elif rate < -2:
#         y = 2
#     else:
#         y = 3
#     print(rate)
#     print(y)
#     print(x.to(T.float32))



(tensor([ 2.3982e+02,  2.4816e+02,  2.3906e+02,  2.4326e+02,  2.4326e+02,
         1.4591e+08,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
         1.4591e+08,         nan,         nan,         nan,         nan,
                nan,         nan,         nan, -1.1438e+07,         nan,
         2.4349e+02,  0.0000e+00]), 5)


In [ ]:
from torch.utils.data import DataLoader
